In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from scipy import stats
from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Вшэ/Риск менеджмент/RM project 1/

/content/drive/MyDrive/Вшэ/Риск менеджмент/RM project 1


In [ ]:
oot = pd.read_csv('task6_oot.csv')
oos = pd.read_csv('task6_test.csv')
train = pd.read_csv('task6_train.csv')

In [ ]:
train.head()

,Unnamed: 0,funded_amnt,train_pred,issue_d
0,0,35000.0,0.065911,2015-07-01
1,1,21000.0,0.440009,2016-02-01
2,2,17575.0,0.460386,2014-01-01
3,3,7000.0,0.006583,2014-07-01
4,4,13000.0,0.006125,2014-05-01


In [ ]:
oot=oot.rename(columns={'oot_pred':'pd'})
oos=oos.rename(columns={'test_pred':'pd'})
train=train.rename(columns={'train_pred':'pd'})

In [ ]:
df_inter=pd.concat([oot,oos,train])

In [ ]:
df_inter = df_inter.loc[df_inter.issue_d>='2017-01-01', ['funded_amnt','pd']]

In [ ]:
df_inter

,funded_amnt,pd
0,10000.0,0.007069
1,12550.0,0.009449
2,20500.0,0.013328
3,24000.0,0.018211
4,5000.0,0.005445
...,...,...
1298848,40000.0,0.044734
1298849,25000.0,0.464248
1298851,12000.0,0.786167
1298852,16000.0,0.103304


In [ ]:
df_inter.funded_amnt.sum()

14521220225.0

In [ ]:
df_uninter = pd.read_csv('task_6.csv')

In [ ]:
df_uninter=df_uninter.rename(columns={'pred':'pd'})
df_uninter = df_uninter.loc[df_uninter.issue_d>='2017-01-01', ['funded_amnt','pd']]
df_uninter

,funded_amnt,pd
6,25000.0,0.009956
10,22000.0,0.005948
12,15600.0,0.003453
16,30000.0,0.011286
22,15000.0,0.796503
...,...,...
2029947,24000.0,0.002911
2029948,3000.0,0.020787
2029949,3200.0,0.047023
2029950,15000.0,0.052022


## Ожидаемые и неожидаемые убытки
Портфель состоящий из *n* кредитов.

Ожидаемые убытки (EL) портфеля за определенный временной промежуток (обычно 1 год):
$$
EL = PD_i * EAD_i * LGD_i
$$

Неожидаемые убытки (UL) портфеля за определенный временной промежуток (иногда эту величину называют кредитным VaR/credit VaR):

$$
UL = VaR_{99.9\%} - EL
$$

Этот показатель отражает степень, в которой EL может быть превышен в экстремальных сценариях за данный период и для заданного доверительного интервала.
Для рассчета неожидаемых убытков необходимо определить распределение убытков портфеля, которое оценивается с помощью однофакторной модели Васичека.

## Модель Васичека

Модель Васичека учитывает 2 причины дефолта: систематический риск (общий фактор экономики/сектора) и идиосинкратический риск конкретного контрагента.
Вероятность дефолта одного кредита выражается через случайную величину $X_i$:

$$
X_i = \sqrt{\rho} * Z + \sqrt{1 - \rho} * \phi_i
$$

where:
   - $\rho$ корреляция между дефолтами всех заёмщиков, отражает влияние систематического риска.
   - $Z$ стандарная нормальная случайная величина, описывающая макроэкономическую ситуацию.
   - $\phi_i$ стандарная нормальная случайная величина, описывающая макроэкономическую ситуацию.  

Кредит *i* считается в дефолте, если $X_i$ меньше расстояния до дефолта: $X_i < C_{thr}$.

### Interpretable model

Expected loss counter

In [ ]:
EAD = df_inter['funded_amnt']
PD = df_inter['pd']

df_inter['EL'] = PD*EAD #LGD=100%
EL_inter = np.sum(df_inter['EL'])

Var_99.9%

In [ ]:
#calculation based on Vasiceka model
N = df_inter.shape[0]

PD = df_inter['pd']
cred_thresholds = norm.ppf(PD)

N_sim = 100
Z = np.random.normal(size=N_sim)
phi = np.random.normal(size=(N_sim, N))

p1 = 0
p2 = 0.06

In [ ]:
#w/o correlation btw defaults
loss_inter_1 = np.zeros(N_sim)

for i in range(N_sim):
    X = np.sqrt(p1) * Z[i] + np.sqrt(1-p1) * phi[i]
    loss_inter_1[i] = np.dot(np.array(X < cred_thresholds), EAD)#LGD=100%

#with correlation btw defaults
loss_inter_2 = np.zeros(N_sim)

for i in range(N_sim):
    X = np.sqrt(p2) * Z[i] + np.sqrt(1-p2) * phi[i]
    loss_inter_2[i] = np.dot(np.array(X < cred_thresholds), EAD)#LGD=100%

Results for interpretable model:

In [ ]:
print('EL_inter = ', EL_inter)

print('\n w/o corr btw defaults:')
print('Var_99.9%_inter (point estimate) = ', loss_inter_1.mean())
ci_inter_1 = norm.interval(confidence=0.999, loc=loss_inter_1.mean(), scale=stats.sem(loss_inter_1))
print('Var_99.9%_inter (confidence interval) = ', ci_inter_1)

print('\n with corr btw defaults:')
print('Var_99.9%_inter (point estimate) = ', loss_inter_2.mean())
ci_inter_2 = norm.interval(confidence=0.999, loc=loss_inter_2.mean(), scale=stats.sem(loss_inter_2))
print('Var_99.9%_inter (confidence interval) = ', ci_inter_2)

EL_inter =  1263331997.2060091

 w/o corr btw defaults:
UL_inter (point estimate) =  1264194066.0
UL_inter (confidence interval) =  (1263014526.2052822, 1265373605.7947178)

 with corr btw defaults:
UL_inter (point estimate) =  1239877328.0
UL_inter (confidence interval) =  (1143234418.8583853, 1336520237.1416156)


#### Final capital (== UL) w/o corr btw defaults for interpretable model:

In [ ]:
print('Point estimate of capital = ', loss_inter_1.mean() - EL_inter)
print('Confidence interval of capital = ', list(ci_inter_1) - EL_inter)

Point estimate of capital =  862068.7939908504
Confidence interval of capital =  [-317471.00072694 2041608.58870864]


#### Final capital (== UL) with corr btw defaults for interpretable model:

In [ ]:
print('Point estimate of capital = ', loss_inter_2.mean() - EL_inter)
print('Confidence interval of capital = ', list(ci_inter_2) - EL_inter)

Point estimate of capital =  18370926.04399085
Confidence interval of capital =  [-8.62139242e+07  1.22955776e+08]


### Uninterpretable model

Expected loss counter

In [ ]:
EAD = df_uninter['funded_amnt']
PD = df_uninter['pd']

df_uninter['EL'] = PD*EAD #LGD=100%
EL_uninter = np.sum(df_uninter['EL'])

Var_99.9%

In [ ]:
#calculation based on Vasiceka model
N = df_uninter.shape[0]

PD = df_uninter['pd']
cred_thresholds = norm.ppf(PD)

N_sim = 100
Z = np.random.normal(size=N_sim)
phi = np.random.normal(size=(N_sim, N))

p1 = 0
p2 = 0.06

In [ ]:
#w/o correlation btw defaults
loss_uninter_1 = np.zeros(N_sim)

for i in range(N_sim):
    X = np.sqrt(p1) * Z[i] + np.sqrt(1-p1) * phi[i]
    loss_uninter_1[i] = np.dot(np.array(X < cred_thresholds), EAD)#LGD=100%

#with correlation btw defaults
loss_uninter_2 = np.zeros(N_sim)

for i in range(N_sim):
    X = np.sqrt(p2) * Z[i] + np.sqrt(1-p2) * phi[i]
    loss_uninter_2[i] = np.dot(np.array(X < cred_thresholds), EAD)#LGD=100%

Results for uninterpretable model:

In [ ]:
print('EL_uninter = ', EL_uninter)

print('\n w/o corr btw defaults:')
print('Var_99.9%_uninter (point estimate) = ', loss_uninter_1.mean())
ci_uninter_1 = norm.interval(confidence=0.999, loc=loss_uninter_1.mean(), scale=stats.sem(loss_uninter_1))
print('Var_99.9%_uninter (confidence interval) = ', ci_uninter_1)

print('\n with corr btw defaults:')
print('Var_99.9%_uninter (point estimate) = ', loss_uninter_2.mean())
ci_uninter_2 = norm.interval(confidence=0.999, loc=loss_uninter_2.mean(), scale=stats.sem(loss_uninter_2))
print('Var_99.9%_uninter (confidence interval) = ', ci_uninter_2)

EL_uninter =  1244184126.6571403

 w/o corr btw defaults:
UL_uninter (point estimate) =  1244312716.75
UL_uninter (confidence interval) =  (1243076921.8452828, 1245548511.6547174)

 with corr btw defaults:
UL_uninter (point estimate) =  1285251836.75
UL_uninter (confidence interval) =  (1197199541.4833343, 1373304132.0166667)


#### Final capital (== UL) w/o corr btw defaults for uninterpretable model:

In [ ]:
print('Point estimate of capital = ', loss_uninter_1.mean() - EL_uninter)
print('Confidence interval of capital = ', list(ci_uninter_1) - EL_uninter)

Point estimate of capital =  128590.09285974503
Confidence interval of capital =  [-1107204.81185746  1364384.99757719]


#### Final capital (== UL) with corr btw defaults for uninterpretable model:

In [ ]:
print('Point estimate of capital = ', loss_uninter_2.mean() - EL_uninter)
print('Confidence interval of capital = ', list(ci_uninter_2) - EL_uninter)

Point estimate of capital =  41067710.092859745
Confidence interval of capital =  [-4.69845852e+07  1.29120005e+08]
